<a href="https://colab.research.google.com/github/ZeleOeO/Artificial-Intelligence-Projects/blob/main/Poem_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import numpy as np

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/irish-lyrics-eof.txt \
    -O /tmp/irish-lyrics-eof.txt

--2023-03-30 10:32:19--  https://storage.googleapis.com/learning-datasets/irish-lyrics-eof.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 108.177.119.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68970 (67K) [text/plain]
Saving to: ‘/tmp/irish-lyrics-eof.txt’

/tmp/irish-lyrics-e 100%[===================>]  67.35K  --.-KB/s    in 0.09s   

2023-03-30 10:32:19 (735 KB/s) - ‘/tmp/irish-lyrics-eof.txt’ saved [68970/68970]



In [3]:
tokenizer = Tokenizer()

data = open("/tmp/irish-lyrics-eof.txt").read()
corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index)+1


In [4]:
input_sentences = []

#[51, 12, 96, 1217, 48, 2, 69]

for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sentences.append(n_gram_sequence)

In [5]:
max_len = max([len(x) for x in input_sentences])

In [6]:
input_sequences = np.array(pad_sequences(input_sentences, maxlen=max_len, padding="pre"))

In [7]:
xs = input_sequences[:,:-1]
labels = input_sequences[:,-1]


In [8]:
ys = keras.utils.to_categorical(labels, num_classes=total_words)

In [9]:
model = keras.Sequential([
    keras.layers.Embedding(total_words, 240, input_length=max_len-1),
    keras.layers.Bidirectional(keras.layers.LSTM(150)),
    keras.layers.Dense(total_words, activation="softmax")
])

adam = keras.optimizers.legacy.Adam(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

model.fit(xs, ys, epochs=100, verbose=1)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
377/377 [==============================] - 9s 7ms/step - loss: 6.6584 - accuracy: 0.0748
Epoch 2/100
377/377 [==============================] - 3s 8ms/step - loss: 5.6977 - accuracy: 0.1197
Epoch 3/100
377/377 [==============================] - 3s 7ms/step - loss: 4.9912 - accuracy: 0.1641
Epoch 4/100
377/377 [==============================] - 2s 7ms/step - loss: 4.6764 - accuracy: 0.1947
Epoch 5/100
377/377 [==============================] - 2s 7ms/step - loss: 3.9113 - accuracy: 0.2546
Epoch 6/100
377/377 [==============================] - 3s 9ms/step - loss: 3.0879 - accuracy: 0.3439
Epoch 7/100
377/377 [==============================] - 3s 7ms/step - loss: 2.6391 - accuracy: 0.4120
Epoch 8/100
377/377 [==============================] - 3s 8ms/step - loss: 2.2940 - accuracy: 0.4699
Epoch 9/100
377/377 [==============================] - 3s 7ms/step - loss: 2.0213 - accuracy: 0.5188
Epoch 10/100
377/377 [==============================] - 3s 7ms/step - loss: 1.7930 - accura

In [16]:
seed_text = "I killed myself"
next_words = 20

for i in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list= pad_sequences([token_list], maxlen=max_len-1, padding="pre")
  predicted = np.argmax(model.predict(token_list), axis=-1)
  output_word=""
  for word, index in tokenizer.word_index.items():
    if index==predicted:
      output_word=word
      break
  seed_text += " "+output_word

print(seed_text)

1/1 [==============================] - 0s 25ms/step
I killed myself to none of my pocket to one night our wedding day come to possessed me life or love them them
